<a href="https://colab.research.google.com/github/dennisovich29/RAG-chatPDF/blob/main/RAG_chatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install --quiet langchain faiss-cpu sentence-transformers pdfminer.six transformers

In [49]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")


In [66]:
# 📅 Upload PDF file
from google.colab import files
uploaded = files.upload()

import io
pdf_file = next(iter(uploaded))
pdf_path = f"/content/{pdf_file}"



Saving lech107 (2).pdf to lech107 (2).pdf


In [67]:
# 📄 Extract text using pdfminer
from pdfminer.high_level import extract_text

def extract_text_from_pdf(file_path):
    return extract_text(file_path)

def chunk_text(text, chunk_size=500, overlap=100):

    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

raw_text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(raw_text)
print(f"✅ Extracted {len(chunks)} chunks from the PDF")



✅ Extracted 282 chunks from the PDF


In [72]:
# 📚 Create vector store using FAISS
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_texts(chunks, embeddings)


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")




In [77]:
# 🔎 Ask a question with manual prompt formatting
context = " ".join(chunks[:100])
question = "What is this document about?"
prompt_text = f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {question}"

inputs = tokenizer(prompt_text, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, max_new_tokens=256)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nQ:", question)
print("A:", answer)


Q: What is this document about?
A: The following is a list of elements of the p-block of periodic table.
